In [1]:
########################################
# ライブラリ
########################################

import jax
import jax.numpy as jnp

from flax import linen as nn
from flax.training import train_state
import optax

import pandas as pd

In [2]:
########################################
# モデルの定義
########################################

class MatrixFactorization(nn.Module):
    
    k: int = 20
    
    @nn.compact
    def __call__(self, user_onehot, item_onehot):
        user_vector = nn.Dense(features=self.k, use_bias=False)(user_onehot)
        item_vector = nn.Dense(features=self.k, use_bias=False)(item_onehot)
        return jnp.sum(user_vector * item_vector, axis=1)

In [3]:
########################################
# 損失関数
########################################

@jax.jit
def loss_function(params, X_USER, X_ITEM, y):
    # 予測値計算
    predict = model.apply({'params': params}, X_USER, X_ITEM)
    # 損失を計算
    loss = jnp.mean( (predict - y)**2 )
    return loss

In [4]:
########################################
# データセットの読み込み
########################################

ML100K = pd.read_table("/home/sugahara/data/ml-100k/row/u.data", header=None)

X_USER = pd.get_dummies(ML100K[0]).values
X_ITEM = pd.get_dummies(ML100K[1]).values
y = ML100K[2].values

X_USER, X_ITEM, y = jax.device_put(X_USER), jax.device_put(X_ITEM), jax.device_put(y)

user_size = X_USER.shape[1]
item_size = X_ITEM.shape[1]

In [5]:
########################################
# モデル
########################################

# モデルの作成
model = MatrixFactorization(k=20)
params = model.init( jax.random.PRNGKey(0), jnp.ones((1, user_size)), jnp.ones((1, item_size)) )["params"]

# SGDを定義
learning_rate = 0.001
tx = optax.adam(learning_rate)

# パラメータの管理
state = train_state.TrainState.create(apply_fn=model.apply, params=params, tx=tx)

In [6]:
########################################
# 学習
########################################

epoch_nums = 512

for epoch_id in range(epoch_nums):
    
    # 損失と勾配を計算
    loss, grads = jax.value_and_grad(loss_function)(state.params, X_USER, X_ITEM, y)

    # 更新
    state = state.apply_gradients(grads=grads)

    # 誤差の確認
    print(
        "訓練誤差:",
        '{:.3f}'.format(loss_function(state.params, X_USER, X_ITEM, y)),
        f"【 Epoch: {epoch_id} / {epoch_nums} 】"
    )

訓練誤差: 13.727 【 Epoch: 0 / 512 】
訓練誤差: 13.726 【 Epoch: 1 / 512 】
訓練誤差: 13.725 【 Epoch: 2 / 512 】
訓練誤差: 13.724 【 Epoch: 3 / 512 】
訓練誤差: 13.724 【 Epoch: 4 / 512 】
訓練誤差: 13.723 【 Epoch: 5 / 512 】
訓練誤差: 13.722 【 Epoch: 6 / 512 】
訓練誤差: 13.721 【 Epoch: 7 / 512 】
訓練誤差: 13.719 【 Epoch: 8 / 512 】
訓練誤差: 13.718 【 Epoch: 9 / 512 】
訓練誤差: 13.716 【 Epoch: 10 / 512 】
訓練誤差: 13.714 【 Epoch: 11 / 512 】
訓練誤差: 13.712 【 Epoch: 12 / 512 】
訓練誤差: 13.710 【 Epoch: 13 / 512 】
訓練誤差: 13.707 【 Epoch: 14 / 512 】
訓練誤差: 13.705 【 Epoch: 15 / 512 】
訓練誤差: 13.701 【 Epoch: 16 / 512 】
訓練誤差: 13.698 【 Epoch: 17 / 512 】
訓練誤差: 13.694 【 Epoch: 18 / 512 】
訓練誤差: 13.690 【 Epoch: 19 / 512 】
訓練誤差: 13.686 【 Epoch: 20 / 512 】
訓練誤差: 13.681 【 Epoch: 21 / 512 】
訓練誤差: 13.676 【 Epoch: 22 / 512 】
訓練誤差: 13.670 【 Epoch: 23 / 512 】
訓練誤差: 13.664 【 Epoch: 24 / 512 】
訓練誤差: 13.658 【 Epoch: 25 / 512 】
訓練誤差: 13.651 【 Epoch: 26 / 512 】
訓練誤差: 13.643 【 Epoch: 27 / 512 】
訓練誤差: 13.635 【 Epoch: 28 / 512 】
訓練誤差: 13.627 【 Epoch: 29 / 512 】
訓練誤差: 13.618 【 Epoch